### Demography
Пример отчета по анализу демографии проектов.

## Описание задания


Задача посчитать аудиторию проекта Kommersant.ru с разбивкой по демографическим переменным:
- Пол
- Пол / Возраст
- Род занятий



Общие параметры:
- Период: Май 2020
- География: Россия
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop,Web Mobile,App Online,App Offline).

В этом шаблоне приведен пример расчета Reach Row%, Reach Column%,
список статистик для расчета:
- Uni
- Unweighted Reach
- Reach
- Reach%
- Reach Row%
- Reach Column%

## Инициализация

Первым шагом любого ноутбука для построения отчетов необходимо загрузить библиотеки, которые нам помогут обращаться к API Responsum и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
#import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging

from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

Loading BokehJS ...

## Формируем задания

Нам понадобится сформировать четыре задания для Responsum:
- расчет аудитории по проекту Kommersant с разбивкой по демографии
- расчет аудитории по населению
- расчет аудитории Total Internet без разбивки по демографии
- расчет аудитории по проекту Kommersant без разбивки по демографии

### Общие параметры для заданий

Для начала зададим общие параметры

In [2]:
# Задаем параметры
# выбираем тип установки mobile, т.к. в расчетах учавствуют все типы устройств (Web Desktop,Web Mobile,App Online,App Offline)
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop-pre'

# создадим объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)


# задаем период расчета
date_from = '2020-05-01'
date_to = '2020-05-31'

# задаем Типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# задаем Население, нас интересует города с населением 100тыс. и больше
population = rcats.get_population('Russia100-')

# задаем Возврастные группы, нас интересует 12+
ages = rcats.get_age_groups('12+')


# Провеяем, что значения парамеров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes, population, ages)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

Дата/время расчета: 2021-03-16 12:11:26
Тип установки: mobile
Начало периода: 2020-05-01
Конец периода: 2020-05-31
Типы пользования Интернетом: [1, 2, 3, 4]
Население: [4]
Возрастные группы: [1, 2, 3, 4, 5, 6, 7]
Объектов в media-каталоге: 4716


## Демографические переменные
Найдем идентификторы демографических переменных для следущих атрибутов:

- Пол
- Пол / Возраст
- Род занятий

Они понадобятся нам для указания разбивки. Для этого воспользуемся блокнотом к котором приведены примеры работы с [каталогами](catalogs.ipynb)

Получили:

- Пол  = 106
- Пол / Возраст = 159
- Род занятий = 175

### Получим ID проектов
Для построения отчета нам необходимо получить идентификатор __Kommerasnt.ru__, для этого воспользуемся блокнотом в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Kommerasnt.ru  site_id = 118

## Задания
Перейдем к формированию заданий

### Задание №1 Статистики по проекту Kommersant с разбивкой по демографии

Расчитаем статистики с разбивкой по проекту и демографии

In [3]:
# Задаем название проекта для отображения в DataFrame
project_name = 'kommersant'

# Задаем медиа фильтр, в нашем случае это ID проекта Kommersant.ru
media_filter = "site = 118"
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета
statistics=["UnwReach", "Reach", "ReachPer"]
# Указываем разбивки, в нашем случае это разбивка по соцдем переменным:
structure =  {
    "media": ["site"],
    "usetype": False,
    "demo": ["SEXAGE", "OCCUPATION2"]
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_kommersant = rtask.result2table(rtask.get_result(task_audience), project_name=project_name, axis_y=['demo'])
df_kommersant

Задача <7f03f9c5-3939-4d91-ac89-c4c0698b3e63> поступила в обработку
Расчет задачи [= = = = = ] время расчета: 0:00:16.614536


12:11:49 INFO:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
12:11:49 INFO:NumExpr defaulting to 8 threads.


,prj_name,attrtitle_demo,attrval_demo,media_site,stat_unwreach,stat_reach,stat_reach_per
0,kommersant,Пол / Возраст,Женщины 25-34,Kommersant.ru,36.0,238.800759,5.465755
1,kommersant,Пол / Возраст,Мужчины 18-24,Kommersant.ru,3.0,39.666995,1.730282
2,kommersant,Пол / Возраст,Женщины 18-24,Kommersant.ru,6.0,71.540489,3.141410
3,kommersant,Пол / Возраст,Мужчины 55-64,Kommersant.ru,34.0,235.311762,5.022192
4,kommersant,Пол / Возраст,Мужчины 25-34,Kommersant.ru,32.0,179.877730,3.466747
5,kommersant,Пол / Возраст,Женщины 65+,Kommersant.ru,13.0,312.034036,4.514382
6,kommersant,Пол / Возраст,Женщины 45-54,Kommersant.ru,41.0,352.966119,7.613866
7,kommersant,Пол / Возраст,Мужчины 45-54,Kommersant.ru,69.0,470.870849,11.014440
8,kommersant,Пол / Возраст,Женщины 55-64,Kommersant.ru,40.0,419.605209,7.247840
9,kommersant,Пол / Возраст,Женщины 12-17,Kommersant.ru,2.0,126.976670,5.731729


### Расчет дополнительных заданий
Нам необходимо посчитать процентные значения для разных срезов (Row%, Col%),
для этого сформируем дополнительные задания и досчитаем необходимые значения

### Задание №2 Демография по всему Internet (без разбивки по media)

In [4]:
# Задаем название проекта для отображения в DataFrame
project_name = 'total_demo'
# Фильтр по медиа отсутствует
media_filter = None
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета
statistics=["Uni", "UnwReach", "Reach", "ReachPer"]
# Указываем порядок разбивки
structure =  {
    "usetype": False,
    "demo": ["SEXAGE", "OCCUPATION2"]
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total_demo = rtask.result2table(rtask.get_result(task_audience), project_name=project_name, axis_y=['demo'])
df_total_demo

Задача <435da113-88ff-4a5d-856a-4a709c783e64> поступила в обработку
Расчет задачи [= = = = = = = ] время расчета: 0:00:23.284893


,prj_name,attrtitle_demo,attrval_demo,stat_unwreach,stat_uni,stat_reach_per,stat_reach
0,total_demo,Пол / Возраст,Женщины 55-64,253.0,5789.382659,58.465059,3384.765992
1,total_demo,Пол / Возраст,Мужчины 25-34,476.0,5188.660584,94.402565,4898.228704
2,total_demo,Пол / Возраст,Мужчины 45-54,388.0,4275.031977,76.847155,3285.240438
3,total_demo,Пол / Возраст,Женщины 35-44,470.0,4956.911597,90.577434,4489.843330
4,total_demo,Пол / Возраст,Женщины 45-54,385.0,4635.833291,82.788777,3837.949702
5,total_demo,Пол / Возраст,Женщины 12-17,44.0,2215.329418,97.539879,2160.829627
6,total_demo,Пол / Возраст,Мужчины 18-24,133.0,2292.515979,95.251351,2183.652446
7,total_demo,Пол / Возраст,Женщины 65+,75.0,6911.998893,33.690948,2328.717973
8,total_demo,Пол / Возраст,Мужчины 65+,92.0,3500.947160,27.919809,977.457753
9,total_demo,Пол / Возраст,Мужчины 12-17,55.0,2325.792629,92.701637,2156.047836


### Задание №3 Статистики по проекту Коммерсант (без разбивки по демографии)

In [5]:
# Задаем название проекта для отображения в DataFrame
project_name = 'total_project'
# Фильтр по медиа отсутствует
media_filter = "(site = 118)"
# Фильтр по демографии отсутствует
demo_filter = None
# Задаем список статистик для расчета
statistics=["UnwReach", "Reach", "ReachPer"]
# Указываем порядок разбивки
structure =  {
    "usetype": False,
    "media": ["site"]
  }

# Формируем из-заданных параметров заданние для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, population_filter=population, ages_filter=ages, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)
# Отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))
# Получаем результат
df_total_prj = rtask.result2table(rtask.get_result(task_audience), project_name=project_name, axis_y=['demo'])
df_total_prj

Задача <cb46655c-ca72-4493-8b8b-96d4835af662> поступила в обработку
Расчет задачи [= = = ] время расчета: 0:00:10.409700


,prj_name,media_site,stat_unwreach,stat_reach,stat_reach_per
0,total_project,Kommersant.ru,401.0,3268.399505,5.598665


# Посчитаем проценты

Для этого воспользуемся методом библиотеки Медиаскоп _calc_row_col_.

На вход методу передаются четыре DataFrame:
- Данные по проекту/проектам с разбивкой по демографии
- TotalInternet с разбивкой по демографии
- Тотал по проекту/проектам без разбивки по демографии

In [6]:
df_result = rtask.calc_row_col(df_kommersant, df_total_prj, df_total_demo)
df_result

,prj_name,attrtitle_demo,attrval_demo,media_site,unwreach,reach,reach_per,unwreach_col_prc,reach_col_prc,uni,unwreach_row_prc,reach_row_prc
0,kommersant,Пол / Возраст,Женщины 25-34,Kommersant.ru,36.0,238.800759,5.465755,8.977556,7.306352,4369.035313,8.845209,5.685254
1,kommersant,Пол / Возраст,Мужчины 18-24,Kommersant.ru,3.0,39.666995,1.730282,0.748130,1.213652,2292.515979,2.255639,1.816543
2,kommersant,Пол / Возраст,Женщины 18-24,Kommersant.ru,6.0,71.540489,3.141410,1.496259,2.188854,2277.337026,6.000000,3.408995
3,kommersant,Пол / Возраст,Мужчины 55-64,Kommersant.ru,34.0,235.311762,5.022192,8.478803,7.199602,4685.439714,15.525114,10.399438
4,kommersant,Пол / Возраст,Мужчины 25-34,Kommersant.ru,32.0,179.877730,3.466747,7.980050,5.503542,5188.660584,6.722689,3.672302
5,kommersant,Пол / Возраст,Женщины 65+,Kommersant.ru,13.0,312.034036,4.514382,3.241895,9.546998,6911.998893,17.333333,13.399391
6,kommersant,Пол / Возраст,Женщины 45-54,Kommersant.ru,41.0,352.966119,7.613866,10.224439,10.799357,4635.833291,10.649351,9.196736
7,kommersant,Пол / Возраст,Мужчины 45-54,Kommersant.ru,69.0,470.870849,11.014440,17.206983,14.406772,4275.031977,17.783505,14.332919
8,kommersant,Пол / Возраст,Женщины 55-64,Kommersant.ru,40.0,419.605209,7.247840,9.975062,12.838247,5789.382659,15.810277,12.396875
9,kommersant,Пол / Возраст,Женщины 12-17,Kommersant.ru,2.0,126.976670,5.731729,0.498753,3.884980,2215.329418,4.545455,5.876293


## Экспорт в Excel

In [7]:
writer = pd.ExcelWriter(rtask.get_excel_filename('demography-kommersant'))
df_info = rtask.get_report_info()
df_result.to_excel(writer, 'Report', index=False)
df_info.to_excel(writer, 'Info', index=False)
writer.save()